In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os

# Устройство (CPU или GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используем устройство: {device}")

# Аугментация данных для улучшения обучения
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # Случайное отражение
    transforms.RandomRotation(15),  # Вращение на ±15 градусов
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Сдвиг до 10%
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),  # Искажение
    transforms.ColorJitter(brightness=0.3, contrast=0.3),  # Яркость и контраст
    # transforms.RandomErasing(p=0.5),  # Удаление случайных фрагментов изображения
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Загрузка датасета
train_dataset = ImageFolder("dataset/train", transform=train_transforms)
val_dataset = ImageFolder("dataset/val", transform=val_transforms)

# Создание DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Проверка классов
print(f"Классы: {train_dataset.classes}")


Используем устройство: cpu
Классы: ['defect', 'no-defect']


In [1]:
import torch.nn as nn
from torchvision import models

# Загрузка предобученной модели ResNet-18
model = models.resnet18(pretrained=True)

# Замена последнего слоя
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Два класса: "дефект" и "без дефекта"

# Перенос модели на устройство
model = model.to(device)

# Функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("Модель подготовлена к обучению")


c:\Users\Alexandr\Documents\Варя\ATP\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Alexandr\Documents\Варя\ATP\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


NameError: name 'device' is not defined

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        print(f"Эпоха {epoch+1}/{num_epochs}")
        
        # Режим обучения
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Средняя потеря на обучении: {running_loss / len(train_loader)}")

        # Оценка на валидации
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Точность на валидации: {accuracy:.2f}%")

# Запуск обучения
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)


Эпоха 1/20
Средняя потеря на обучении: 0.29079421383461784
Точность на валидации: 71.43%
Эпоха 2/20
Средняя потеря на обучении: 0.17214264401367732
Точность на валидации: 85.71%
Эпоха 3/20
Средняя потеря на обучении: 0.08412256261466869
Точность на валидации: 80.36%
Эпоха 4/20
Средняя потеря на обучении: 0.07005584545965705
Точность на валидации: 85.71%
Эпоха 5/20
Средняя потеря на обучении: 0.08268185531986612
Точность на валидации: 75.00%
Эпоха 6/20
Средняя потеря на обучении: 0.05370537457721574
Точность на валидации: 85.71%
Эпоха 7/20
Средняя потеря на обучении: 0.06206293665205261
Точность на валидации: 85.71%
Эпоха 8/20
Средняя потеря на обучении: 0.04857733752578497
Точность на валидации: 87.50%
Эпоха 9/20
Средняя потеря на обучении: 0.05211709833071966
Точность на валидации: 83.93%
Эпоха 10/20
Средняя потеря на обучении: 0.05168145386103008
Точность на валидации: 87.50%
Эпоха 11/20
Средняя потеря на обучении: 0.08419819896308971
Точность на валидации: 83.93%
Эпоха 12/20
Средняя

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Точность на валидации: {accuracy:.2f}%")

Точность на валидации: 85.71%


In [ ]:
import cv2
import numpy as np
import os

def remove_red_triangle(image_path, save_path):
    # Загружаем изображение
    image = cv2.imread(image_path)
    if image is None:
        print(f"Ошибка загрузки {image_path}")
        return

    # Переводим в формат HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Диапазон красного цвета (можно подстроить)
    lower_red1 = np.array([0, 120, 70])   # Красный оттенок 1
    upper_red1 = np.array([10, 255, 255]) 
    lower_red2 = np.array([170, 120, 70]) # Красный оттенок 2
    upper_red2 = np.array([180, 255, 255])

    # Маска красного цвета
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = mask1 + mask2  # Объединяем две маски

    # Находим контуры красных объектов
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Замазываем красные области
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        roi = image[y:y+h, x:x+w]  # Выделяем область
        blurred = cv2.GaussianBlur(roi, (15, 15), 0)  # Размываем
        image[y:y+h, x:x+w] = blurred  # Заменяем

    # Сохраняем результат
    cv2.imwrite(save_path, image)
    print(f"Обработанное изображение сохранено в {save_path}")

# Пример использования
remove_red_triangle("dataset/val/defect/photo_2025-02-02_18-47-09.jpg", "new_photo.jpg")

Ошибка загрузки dataset/val/defect/photo_2025-02-02_18-47-09.jpg


In [2]:
from torch import save

save(model.state_dict(), "defect_model.pth")